## 59. 队列的最大值

题目一：滑动窗口的最大值

给定一个数组和滑动窗口的大小，请找出所有滑动窗口里的最大值。例如：如果输入数组{2,3,4,2,6,2,5,1}及滑动窗口的大小3，那么一共存在6个滑动窗口，它们的最大值分别是{4,4,6,6,6,5}



### 分析
蛮力法：如果滑动窗口的大小为k，则需要$O(k)$时间找出窗口里的最大值，对于总长度为n的输入数组，这种算法的总时间复杂度$O(nk)$

实际上，一个滑动窗口可以看作一个队列，每次删除头一个数，尾部加入另一个数。在**面试题30**中，我们实现了一个可以用$O(1)$时间得到最小值的栈。在**面试题9**中，我们讨论了如何用两个栈实现一个队列。总结起来，如果用两个栈实现一个队列，我们可以用$O(1)$时间得到队列中的最大值，则把总的时间复杂度降到了$O(n)$


另一种思路：

我们并不把滑动窗口的每个数值都存入队列，而是只把有可能成为滑动窗口最大值的数值存入一个两端开口的队列。以{2,3,4,2,6,2,5,1}为例：


- 第一个数字2，存入队列，第二个数字3，比2大 --> 把3存入队列并删除2。针对第三个数字4进行类似的操作，此时到达了`window size = 3`，输出`4`。4处于队列的**头部**。

- 接下来的数字是2，2比队列中的数字4小，当4滑出窗口后，2还是有可能成为滑动窗口中的最大值，因此把2存入队列的**尾部**。4比2大，输出4

- 第五个数字是6，6>4>2，4和2已经不可能成为滑动窗口中的最大值，把4，2从队列中删除，把6存入队列，位于队列**头部**。输出6

- 第六个数字是2，存入队列尾部，输出6。

- 第七个数字5，5>2, 把2从尾部删除，把5存入尾部，输出6.

- 第八个数字1，注意此时6已经不在这个sliding window里面了，如何才能知道某个元素不在sliding window里面？所以我们其实应该在队列里存原数组的index才对。

[//]: # (<img src="images/img123.png" style="width: 500px;"/>)

In [1]:
def max_in_sliding_window(arr: list, ws: int):
    if arr is None or arr == [] or ws > len(arr):
        return []

    result = []
    two_side_queue = [0]
    for i in range(1, len(arr)):
        if i >= ws:
            result.append(arr[two_side_queue[0]])

        # check if any element in two_side_queue in out of current sliding window
        for idx in two_side_queue:
            if i - idx >= ws:
                two_side_queue.remove(idx)

        if arr[i] > arr[two_side_queue[0]]:
            two_side_queue[0] = i
            continue
        if len(two_side_queue) == 2:
            if arr[i] > arr[two_side_queue[1]]:
                two_side_queue[1] = i
        else:
            two_side_queue.append(i)

    result.append(arr[two_side_queue[0]])

    return result

# TEST
arr = [2, 3, 4, 2, 6, 2, 5, 1]
print(max_in_sliding_window(arr, 3))

[4, 4, 6, 6, 6, 5]


### 题目二：队列的最大值
请定义一个队列并实现函数max得到队列里的最大值，要求函数max、push_back、pop_front的时间复杂度都是$O(1)$

### 分析
如前所述，sliding window可以看成是一个队列，因此上题的解法可以用来实现带max函数的队列。

In [2]:
class InternalData:
    def __init__(self, number, index):
        self.number = number
        self.index = index

class QueueWithMax:
    def __init__(self):
        self.data = []
        self.maximums = []
        self.currentIndex = 0

    def push_back(self, number):
        while len(self.maximums) > 0 and number >= self.maximums[-1].number:
            self.maximums.pop()

        internalData = InternalData(number, self.currentIndex)
        self.data.append(internalData)
        self.maximums.append(internalData)

        self.currentIndex += 1

    def pop_front(self):
        if len(self.maximums) == 0:
            Exception("queue is empty")

        if self.maximums[0].index == self.data[0].index:
            self.maximums.pop(0)

        self.data.pop(0)

    def max(self):
        if len(self.maximums) > 0:
            Exception("queue is empty")

        return self.maximums[0].number